In [68]:
from numba.experimental import jitclass
from numba import njit
from numba.core import types
from numba import float64
import os
import sys
import pandas as pd

sys.dont_write_bytecode = True
# os.environ["NUMBA_DISABLE_JIT"] = "1"
os.environ["NUMBA_DEBUG_CACHE"] = "1"

In [56]:
# https://numba.discourse.group/t/typed-list-of-jitted-functions-in-jitclass/413/6


# class HelperClass:
#     def __init__(self, helper_class_self_num: float, helper_class_self_var: float) -> None:
#         self.helper_class_self_num = helper_class_self_num
#         self.helper_class_self_var = helper_class_self_var

#     def calc_num_one(self, helper_num: float):
#         pass

#     def calc_num_two(self, helper_num: float):
#         pass


# @jitclass([("helper_class_num", types.float_), ("helper_class_var", types.float_)])
spec = {
    "helper_class_self_num": types.float_,
    "helper_class_self_var": types.float_,
}


@jitclass(spec)
class HelperClass(object):
    def __init__(
        self,
        helper_class_self_num,
        helper_class_self_var,
    ):
        self.helper_class_self_num = helper_class_self_num
        self.helper_class_self_var = helper_class_self_var

    def calc_num_one(self, helper_num):
        pass

    # def calc_num_two(self, helper_num: float):
    #     pass


# @jitclass([("helper_class_num", types.float_), ("helper_class_var", types.float_)])
# @jitclass([("helper_class_num", types.float_)])
@jitclass
class SubClassOne(object):
    helper_class_inst: HelperClass
    # helper_class_self_num: types.float_

    def __init__(
        self,
        helper_class_self_num,
        helper_class_self_var,
    ):
        # self.helper_class_init = HelperClass(helper_class_self_num, helper_class_self_var)
        self.helper_class_inst = HelperClass(
            helper_class_self_num,
            helper_class_self_var,
        )
        # print(self.helper_class_inst.helper_class_self_num)
        # self.helper_class_self_num = self.helper_class_inst.helper_class_self_num

    def calc_num_one(self, helper_num):
        return helper_num / self.helper_class_inst.helper_class_self_num

    # def calc_num_two(self, helper_num: float):
    #     return helper_num - self.helper_class_inst.helper_class_self_var


# @jitclass
# class ClassTwo(object):
#     helper_class_inst: HelperClass

#     def __init__(self, helper_class_inst) -> None:
#         self.helper_class_inst = helper_class_data

#     def calc_num_one(self, helper_num: float):
#         return helper_num / self.helper_class_inst.helper_class_self_num

# def calc_num_two(self, helper_num: float):
#     return helper_num - self.helper_class_inst.helper_class_self_var

In [71]:
spec = {
    "helper_class_self_num": types.float_,
    "helper_class_self_var": types.float_,
}

@jitclass(spec)
class HelperClass(object):
    def __init__(
        self,
        helper_class_self_num,
        helper_class_self_var,
    ):
        self.helper_class_self_num = helper_class_self_num
        self.helper_class_self_var = helper_class_self_var

    def calc_num_one(self, helper_num):
        pass


@jitclass
class SubClassOne(object):
    helper_class_inst: HelperClass

    def __init__(
        self,
        helper_class_self_num,
        helper_class_self_var,
    ):
        self.helper_class_inst = HelperClass(
            helper_class_self_num,
            helper_class_self_var,
        )

    def calc_num_one(self, helper_num):
        return helper_num / self.helper_class_inst.helper_class_self_num


class SubTwo(object):
    helper_class_inst: HelperClass

    def __init__(
        self,
        helper_class_self_num,
        helper_class_self_var,
    ):
        self.helper_class_inst = HelperClass(
            helper_class_self_num,
            helper_class_self_var,
        )

    def calc_num_one(self, helper_num):
        return pd.DataFrame([helper_num])


@njit(cache=True)
def tester_func():
    calculator = SubClassOne(10, 20)
    return calculator.calc_num_one(40)


# @njit(cache=True)
# def tester_func():
#     calculator = SubClassOne(helper_class_self_num=10, helper_class_self_var=20)
#     return calculator.calc_num_one(40)


tester_func()

[cache] index saved to 'C:\\Users\\User\\.ipython\\numba_cache\\1102278571.tester_func-fc97a74e06.py310.nbi'
[cache] data saved to 'C:\\Users\\User\\.ipython\\numba_cache\\1102278571.tester_func-fc97a74e06.py310.1.nbc'


4.0

In [72]:
SubTwo(10, 20).calc_num_one(3)

,0
0,3


In [50]:
@njit(cache=True)
def tester_func():
    calculator = SubClassOne(
        helper_class_self_num=10,
        helper_class_self_var=20,
    )
    return calculator.calc_num_one(40)


tester_func()

6.0

In [17]:
import numpy as np
from numba import int64, types, njit
from numba.experimental import jitclass


# spec_foo = {'data': int64}


@jitclass([("data", types.float_)])
class Foo(object):
    def __init__(self, init_data):
        self.data = init_data

    def calc(self, num):
        pass


@jitclass
class Bar(object):
    foo_inst: Foo

    def __init__(self, foo_data):
        # This constructs the Foo instance in the Bar constructor
        self.foo_inst = Foo(foo_data)

    def calc(self, num):
        return num * self.foo_inst.data


@jitclass
class Baz(object):
    foo_inst: Foo

    def __init__(self, foo_inst):
        # This assigns a reference to the Foo instance in the Bar constructor
        self.foo_inst = foo_inst


@njit(cache=True)
def testing_yes():
    x = 30.0
    # Build a Bar instance
    bar = Bar(x)
    print(bar.foo_inst.data)
    print(bar.calc(10))


testing_yes()

# # Create a Baz instance that holds a reference to Bar's Foo instance.
# baz = Baz(bar.foo_inst)

# # Do some arithmetic...
# baz.foo_inst.data += 1
# print(baz.foo_inst.data)
# print(bar.foo_inst.data)

30.0
300.0


In [ ]:
# @njit(cache=True)
# def nb_remander(nb_func_num: float):
#     return nb_func_num % 3.0


# @njit(cache=True)
# def nb_power(nb_func_num: float):
#     return nb_func_num**2.0
# @numba.experimental.jitclass(
#     [
#         # ("helper_class", nb_HelperClass.class_type.instance_type),
#         ("nb_func", types.float_(types.float_).as_type()),
#         ("jit_test_num", types.float_),
#     ]
# )
# class JitTest(object):
#     helper_class_inst: HelperClass

#     def __init__(self, helper_class_inst, nb_func, jit_test_num):
#         self.helper_class_inst = helper_class_inst
#         self.nb_func = nb_func
#         self.jit_test_num = jit_test_num

#     def final_calc(self, helper_num: float, nb_func_num: float):
#         return (
#             self.helper_class_inst.calc_num_two(helper_num=helper_num)
#             + self.nb_func(nb_func_num=nb_func_num)
#             + self.jit_test_num
#         )


# @njit(cache=True)
# def exec_nb_class():
#     do_jit_test = JitTest(
#         ClassTwo(helper_class_self_num=10.0, helper_class_self_var=20.0),
#         nb_power,
#         6.0,
#     )
#     return do_jit_test.final_calc(helper_num=30.0, nb_func_num=2.0)


# exec_nb_class()
"""
do_jit_test.final_calc(helper_num = 30.0, nb_func_num = 3.0)
    self.helper_class.calc_num_two(helper_num = 30.0) + self.nb_func(nb_func_num = 2) + self.jit_test_num
        self.helper_class.calc_num_two(helper_num = 30.0)
            ClassTwo().calc_num_two(helper_num = 30.0)
                helper_num - self.helper_class_self_var
                30.0 - 20.0 = 10.0

        self.nb_func(nb_func_num = 3.0)
            nb_power(nb_func_num = 3.0)
            3.0**2.0 = 4.0

        self.class_num = 6.0

        10.0 + 4.0 + 6.0 = 20.0

        do_jit_test.final_calc(helper_num = 30.0, nb_func_num = 2.0) = 20.0

exec_nb_class() = 20.0

"""

In [ ]:
@jitclass